# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [1]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [2]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [3]:
os.environ["RAGAS_APP_TOKEN"] = getpass.getpass("Please enter your Ragas API key!")

In [4]:
import os
from langsmith import Client
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager

# Set up LangSmith environment
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API Key:")
os.environ["LANGCHAIN_PROJECT"] = "retriever_comparison"

# Create callback manager
langsmith_client = Client()
tracer = LangChainTracer(project_name="retriever_comparison")
callback_manager = CallbackManager([tracer])

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [5]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-03-01 21:38:09--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[2606:50c0:8003::154]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 19628 (19K) [text/plain]
Guardando como: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19,17K  --.-KB/s    en 0s      

2025-03-01 21:38:10 (69,4 MB/s) - ‘john_wick_1.csv’ guardado [19628/19628]

--2025-03-01 21:38:10--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[2606:50c0:8003::154]:443... conectado.
Petición HTTP enviada, esper

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [7]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 2, 26, 21, 38, 12, 352302)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [8]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [9]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [11]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [12]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [13]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [14]:
res = naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"}, {"tags": ["naive_retriever"]})

In [15]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. This sets off a chain of events where he unleashes destruction against those who try to stop him. It's a story filled with action, suspense, and intense fights."

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [16]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [17]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [18]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Overall, opinions on the John Wick movies seem to be mixed. Some people really enjoy the action and style of the films, while others find them lacking in substance and plot. It ultimately depends on personal preferences.'

In [19]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I'm sorry, but the reviews provided do not have a rating of 10."

In [20]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the protagonist, portrayed by Keanu Reeves, is a retired hitman seeking vengeance for the death of his puppy, a final gift from his deceased wife. The movie is known for its beautifully choreographed action sequences and emotional depth despite being categorized as an action flick.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [21]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever)

Let's create our chain again, and see how this does!

In [22]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, it seems that people generally liked the first John Wick movie. The reviews mention that it was a remarkable, surprising film that caught viewers off guard and that it was something special with elaborate action sequences.'

In [24]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'."

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, after resolving his issues with the Russian mafia, John Wick returns home. However, the mobster Santino D'Antonio visits him with his marker asking for help. John Wick refuses, and Santino blows up his house. To honor the marker, John Wick is asked to kill Santino's sister in Rome. Once he completes the task, a contract is put on him, attracting professional killers from all over. Wick promises to kill Santino in revenge."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [26]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [27]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [28]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews provided.'

In [29]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I don't know."

In [30]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, Keanu Reeves plays the character of John Wick, a retired assassin who comes out of retirement seeking revenge after some Russian mobsters steal his car, kill his dog, and inadvertently push him back into the world of assassins. The story follows John as he goes on a vengeful killing spree against those who wronged him.'

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [31]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [32]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/tmp/ipykernel_69029/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [33]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter
)

By default, this is empty as we haven't added any documents - let's add some now!

In [34]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [35]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [36]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Some people liked John Wick, while others did not. It seems to have received mixed reviews based on the opinions shared.'

In [37]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\'.'

In [38]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the protagonist, played by Keanu Reeves, is a retired assassin who comes out of retirement after his dog is killed in the first movie. In the second movie, he is forced back into the world of assassination when an Italian crime lord calls in a favor. Wick then goes on a violent journey to fulfill this task, traveling to Italy, Canada, and Manhattan, and killing numerous assassins along the way.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [39]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [40]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [41]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, it seems that in general, people liked John Wick. The movie received positive reviews praising the action sequences, Keanu Reeves' performance, and the world-building aspect of the film. Overall, the feedback from the reviews suggests that the movie was well-received by the audience."

In [42]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Reviewer: ymyuseda\n- Rating: 10\n- URL: [Link to Review](https://www.imdb.com/review/rw4854296/?ref_=tt_urv)'

In [43]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," an ex-hit-man comes out of retirement to track down the gangsters that killed his dog and took everything from him. With the untimely death of his beloved wife still bitter in his mouth, he seeks vengeance. Wick finds himself dragged into an impossible task as every killer in the business dreams of cornering him. As he goes on a rampage seeking revenge, he becomes the target of hit men and bounty-hunting killers. The plot revolves around his relentless vendetta against those who have wronged him.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [44]:
#!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [45]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [46]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [47]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [48]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [49]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [50]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, it seems that people generally like John Wick.'

In [51]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\''

In [52]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is a movie about a retired assassin named John Wick seeking revenge on those who killed his dog and stole his car. It is a beautifully simple premise for an action movie filled with awesome action, stylish stunts, and a relatable hero.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [53]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())




In [54]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(documents, testset_size=10)

Applying SummaryExtractor:   0%|          | 0/44 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/100 [00:00<?, ?it/s]

Node 21f344b0-ccca-4416-9615-c841196a5f9c does not have a summary. Skipping filtering.
Node 9f822eca-4e3d-4789-a2d2-cad27cef583d does not have a summary. Skipping filtering.
Node fc89e06b-eb12-4896-a715-1afcd2f72bab does not have a summary. Skipping filtering.
Node af5f3d3d-0ff1-4595-bc71-05ffb446432c does not have a summary. Skipping filtering.
Node d37c2185-3f8c-48f2-b1f3-bd0e66071170 does not have a summary. Skipping filtering.
Node a62ad3ec-b243-4306-8c08-b88c3f1942ee does not have a summary. Skipping filtering.
Node 3a9292f6-10db-4113-b819-927129cf909e does not have a summary. Skipping filtering.
Node e02b6b2d-2414-4714-aeeb-c10bbc8c376d does not have a summary. Skipping filtering.
Node 1e467949-568b-48c0-8173-51cadd13e965 does not have a summary. Skipping filtering.
Node fb84941a-7d64-43f2-8e44-694f177d64ca does not have a summary. Skipping filtering.
Node 2fa9d1b8-1721-4318-a18f-338c3743e5f9 does not have a summary. Skipping filtering.
Node 23199e27-3719-49af-a53b-ff415eae9d97 d

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/244 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

### Ragas Metrics

In [55]:
def chain_evaluation(dataset,chain, tag):
    from ragas import evaluate
    from ragas.llms import LangchainLLMWrapper
    from ragas import EvaluationDataset

    from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
    from ragas import evaluate, RunConfig

    for test_row in dataset:
        response = chain.invoke({"question" : test_row.eval_sample.user_input}, {"tags": [tag]})
        test_row.eval_sample.response = response["response"].content
        test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]


    evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

    evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

    custom_run_config = RunConfig(timeout=360)

    result = evaluate(
        dataset= evaluation_dataset,
        metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
        llm= evaluator_llm,
        run_config=custom_run_config
    )
    return result

In [56]:

naive_result = chain_evaluation(dataset,naive_retrieval_chain,'naive_retriever')


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [57]:
naive_result

{'context_recall': 0.8333, 'faithfulness': 0.8248, 'factual_correctness': 0.2690, 'answer_relevancy': 0.5718, 'context_entity_recall': 0.4567, 'noise_sensitivity_relevant': 0.4914}

In [58]:
bm25_result = chain_evaluation(dataset,bm25_retrieval_chain,'bm25_retriever')

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [59]:
bm25_result

{'context_recall': 0.4333, 'faithfulness': 0.6778, 'factual_correctness': 0.2720, 'answer_relevancy': 0.5814, 'context_entity_recall': 0.3117, 'noise_sensitivity_relevant': 0.3118}

In [60]:
contextual_compression_result = chain_evaluation(dataset,contextual_compression_retrieval_chain,'contextual_compression_retriever')



Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [61]:
multi_query_result = chain_evaluation(dataset,multi_query_retrieval_chain, 'multi_query_retriever')


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Exception raised in Job[47]: TimeoutError()


In [62]:
parent_document_result = chain_evaluation(dataset,parent_document_retrieval_chain,'parent_document_retriever')


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [63]:
ensemble_result = chain_evaluation(dataset,ensemble_retrieval_chain,'ensemble_retriever')


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [64]:
semantic_result = chain_evaluation(dataset,semantic_retrieval_chain,'semantic_retriever')

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [83]:
import pandas as pd

# Convert dictionary to DataFrame
df = pd.read_csv('Evaluation_Metrics.csv')

df.rename(columns={'Unnamed: 0': 'Retriever'}, inplace=True)

display(df)

,Retriever,context_recall,faithfulness,factual_correctness,answer_relevancy,context_entity_recall,noise_sensitivity_relevant
0,Naive,0.8333,0.8248,0.269,0.5718,0.4567,0.4914
1,BM25,0.4333,0.6778,0.272,0.5814,0.3117,0.3118
2,Contextual Compression,0.4833,0.7387,0.368,0.4768,0.4383,0.3639
3,Multi Query,0.9667,0.6557,0.345,0.6746,0.4583,0.6188
4,Parent Document,0.4917,0.7083,0.237,0.4784,0.2650,0.2583
5,Ensemble,0.8667,0.6650,0.361,0.4692,0.3917,0.5830
6,Semantic,0.8583,0.5367,0.268,0.5745,0.3917,0.5958


## Cost and Latency using Langsmith



In [95]:
from langsmith import Client

# Initialize the client
client = Client()

# Function to get runs by tag with corrected filter syntax
def get_runs_by_tag(tag, project_name="retriever_comparison", limit=10):
    try:
        # List runs with the specified tag using the correct filter syntax
        runs = client.list_runs(
            project_name=project_name,
            filter=f'has(tags, "{tag}")'
        )
        return list(runs)  # Convert iterator to list
    except Exception as e:
        print(f"Error retrieving runs: {str(e)}")
        return []

# Get runs for each retriever type
retriever_types = ["naive_retriever", "bm25_retriever", "contextual_compression_retriever", 
                  "multi_query_retriever", "parent_document_retriever", 
                  "ensemble_retriever", "semantic_retriever"]

# Create a dictionary to store results
results = {}

for tag in retriever_types:
    runs = get_runs_by_tag(tag)
    
    if runs:
        
        # Calculate average latency and total cost
        total_latency = 0
        total_cost = 0
        count = 0
        
        for run in runs:
            # Calculate latency
            if hasattr(run, 'end_time') and hasattr(run, 'start_time'):
                latency = (run.end_time - run.start_time).total_seconds()
                total_latency += latency
                count += 1
            
            # Extract cost information - directly from the total_cost attribute
            if hasattr(run, 'total_cost') and run.total_cost is not None:
                total_cost += float(run.total_cost)
        
        # Calculate averages
        avg_latency = total_latency / count if count > 0 else 0
        avg_cost = total_cost / count if count > 0 else 0
        
        # Store results
        results[tag] = {
            'avg_latency': avg_latency,
            'total_cost': total_cost,
            'avg_cost': avg_cost,
            'run_count': count
        }
        print("Metrics for ", tag)
        print(f"Average Latency: {avg_latency:.2f} seconds")
        print(f"Total Cost: ${total_cost:.6f}")
        print(f"Average Cost per Run: ${avg_cost:.6f}")
        print("-" * 85)
    else:
        print(f"No runs found with tag '{tag}'")

# Print summary table
print("\n--- RETRIEVER COMPARISON SUMMARY ---")
print(f"{'Retriever Type':<30} {'Avg Latency (s)':<15} {'Avg Cost ($)':<15} {'Total Cost ($)':<15} {'Run Count':<10}")
print("-" * 85)
for tag, data in results.items():
    print(f"{tag:<30} {data['avg_latency']:<15.2f} {data['avg_cost']:<15.6f} {data['total_cost']:<15.6f} {data['run_count']:<10}")


Metrics for  naive_retriever
Average Latency: 0.54 seconds
Total Cost: $0.073190
Average Cost per Run: $0.000523
-------------------------------------------------------------------------------------
Metrics for  bm25_retriever
Average Latency: 0.36 seconds
Total Cost: $0.030544
Average Cost per Run: $0.000218
-------------------------------------------------------------------------------------
Metrics for  contextual_compression_retriever
Average Latency: 0.73 seconds
Total Cost: $0.027270
Average Cost per Run: $0.000182
-------------------------------------------------------------------------------------
Metrics for  multi_query_retriever
Average Latency: 0.91 seconds
Total Cost: $0.104331
Average Cost per Run: $0.000497
-------------------------------------------------------------------------------------
Metrics for  parent_document_retriever
Average Latency: 0.46 seconds
Total Cost: $0.014522
Average Cost per Run: $0.000104
-----------------------------------------------------------

## **Performance Analysis**

### **Cost Efficiency**
The **Parent Document Retriever** has the lowest cost both per retrieval (**$0.000104**) and total (**$0.014522**), making it the most budget-friendly option.

### **Latency**
The **BM25 Retriever** has the lowest average latency (**0.36s**), making it the fastest retriever.

### **Performance**
- **Best Context Recall**: **Multi Query Retriever** (**0.9667**), meaning it retrieves the most relevant contextual information.
- **Best Faithfulness**: **Naive Retriever** (**0.8248**), generating responses most aligned with retrieved content.
- **Best Factual Correctness**: **Contextual Compression Retriever** (**0.368**), ensuring higher factual accuracy.
- **Best Answer Relevancy**: **Multi Query Retriever** (**0.6746**), producing the most relevant responses to user queries.
- **Best Context Entity Recall**: **Multi Query Retriever** (**0.4583**), capturing the required entities more effectively.
- **Least Noise Sensitivity**: **Parent Document Retriever** (**0.2583**), being less affected by irrelevant context.

---

## **Best Choice**
- If **cost** is the main concern, **Parent Document Retriever** is the best choice.
- If **speed** is the priority, **BM25 Retriever** is the fastest option.
- For the **best overall balance of accuracy, relevancy, and contextual recall**, **Multi Query Retriever** is the most effective, despite being more expensive and slower.
- If **factual correctness** is key, **Contextual Compression Retriever** offers the best balance between accuracy and faithfulness.
- If **a compromise between cost and performance** is needed, **Contextual Compression Retriever** is a solid choice.
